In [18]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from PIL import Image
import io

from datasets import imagenet
from nets import inception
from preprocessing import inception_preprocessing

from tensorflow.contrib import slim

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

image_size = inception.inception_resnet_v2.default_image_size
checkpoints_dir = '/home/src/goodsdl/train/23'



In [19]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
pre_graph = tf.Graph()
with pre_graph.as_default():
    image_path = tf.placeholder(dtype=tf.string,name='input_image')
    image_string = tf.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3, name='image_tensor')
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.expand_dims(image, 0)
    image = tf.image.resize_bilinear(image, [image_size, image_size], align_corners=False)
    image = tf.squeeze(image, [0])
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0, name='output_image')

pre_sess = tf.Session(graph=pre_graph, config=config)

    
graph = tf.Graph()
with graph.as_default():    
    images = tf.placeholder(dtype=tf.float32,shape=[None,image_size,image_size,3],name='input_tensor')
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_resnet_v2_arg_scope()):
        logits, _ = inception.inception_resnet_v2(images, num_classes=156, is_training=False)
    probabilities = tf.nn.softmax(logits,name='detection_classes')
    
    init_fn = slim.assign_from_checkpoint_fn(
        tf.train.latest_checkpoint(checkpoints_dir),
        slim.get_model_variables('InceptionResnetV2'))
    
    sess = tf.Session(config=config)
    init_fn(sess)

INFO:tensorflow:Restoring parameters from /home/src/goodsdl/train/23/model.ckpt-21992


In [20]:
input_image_tensor = pre_graph.get_tensor_by_name('input_image:0')
output_image_tensor = pre_graph.get_tensor_by_name('output_image:0')
input_images_tensor = graph.get_tensor_by_name('input_tensor:0')
detection_classes = graph.get_tensor_by_name('detection_classes:0')
image_dir = '/home/src/goodsdl/train/23/step2/6925303721367'

filelist = [os.path.join(image_dir, image_name) for image_name in os.listdir(image_dir)]

images = [pre_sess.run(output_image_tensor,feed_dict={input_image_tensor:fname}) for fname in filelist]
# print(images[0].shape)

image_nps = np.array(images)
# print(image_nps.shape)

probabilities = sess.run(detection_classes,feed_dict={input_images_tensor:image_nps})
print(probabilities[0])
for i in range(probabilities.shape[0]):
    sorted_inds = [j[0] for j in sorted(enumerate(-probabilities[i]), key=lambda x:x[1])]
    for k in range(5):
        index = sorted_inds[k]
        print('%d:Probability %0.2f%% => [%s]' % (i, probabilities[i][index] * 100, index))
    print('')

[  5.70835588e-13   4.42675713e-07   6.42341735e-09   8.03442989e-12
   1.03724611e-08   6.00965996e-06   9.73319839e-05   1.09279907e-09
   4.59475652e-11   3.44988916e-06   7.81975096e-10   3.61022545e-10
   1.13206465e-10   1.09740171e-07   4.75558481e-09   9.53347290e-10
   1.58333124e-09   1.05723963e-09   8.32752339e-11   2.40559972e-09
   1.85444169e-05   1.09776110e-12   8.77307775e-12   3.59809775e-07
   1.41432210e-09   2.28505773e-11   1.00546047e-11   1.35764493e-08
   2.09500021e-11   1.50273021e-08   2.59040389e-09   4.39807373e-08
   5.15524112e-09   6.43817846e-11   3.13135700e-07   2.31715536e-10
   7.45567732e-08   3.41883921e-12   1.11520157e-12   4.26012292e-09
   7.19592117e-06   7.63210508e-08   1.96055305e-09   2.10563105e-08
   4.28198810e-09   2.64709227e-10   3.66534927e-11   1.03847798e-11
   2.21565633e-09   5.97273186e-12   1.25390170e-06   1.07579019e-07
   1.78051920e-10   3.02978888e-13   1.44726613e-08   2.74070544e-07
   1.13288397e-05   1.00507561e-10